In [1]:
# Global Config
from common.GlobalConfig import *
# Global Util
from common.Util import *
# System
import time
import os
import random
# Statistics
import numpy as np
import pandas as pd
# Graph
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
# Bayesian Optimization
from bayes_opt import BayesianOptimization
# Pytorch
import torch
from torch import nn
import torch.nn.functional as F
import tqdm
from torch.utils.data import DataLoader, TensorDataset
# Model
from models.CNN import cnn

# Create folder if not exist
if not os.path.exists('./best_model'):
    os.makedirs('best_model')
# To show wider screen for output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

max_cnn_acc = 0.8346
max_random_acc = 0.8183
max_bay_acc = 0.8346

/tmp/ipykernel_84185/2593268057.py:30: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# 1. Helper Function

In [2]:
def train_cnn(hidden_size, dropout_rate, learning_rate, l2_reg, num_filters, pool_size, num_mlp_layers, num_cnn_layers, stride=1, kernel_index=0, sigmoid=0, num_of_epochs=5, verbose=False, plot=False, verbose_itr=False, method='bay'):
    # Build Model
    kernel_size = kernel_sizes_map[int(kernel_index)-1]
    sigmoid = True
#     print(hidden_size, dropout_rate, learning_rate, l2_reg, num_filters, pool_size, num_mlp_layers, num_cnn_layers, kernel_index)
    if not check_valid_cnn_output_size(init_size=10, n_layers=int(num_cnn_layers), kernel_sizes=kernel_size, pool_size=int(pool_size), stride=int(stride)):
        return 0
    model = cnn(input_length=10, vocab_size=9, hidden_size=int(hidden_size), kernel_sizes=kernel_size,
                     dropout_rate=dropout_rate, num_filters=int(num_filters), num_mlp_layers=int(num_mlp_layers), output_size=output_size,
                     num_cnn_layers=int(num_cnn_layers), pool_size=int(pool_size), stride=int(stride), sigmoid=sigmoid)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg)
    criterion = get_loss_function('BCE')
    model.to(device)
    total_train_acc=[]
    total_valid_acc=[]
    global max_cnn_acc
    global max_random_acc
    global max_bay_acc
    model.train()
    for e in range(int(num_of_epochs)):    
        if verbose_itr:
            print('Epoch', e+1, flush=True)
        train_acc = 0
        train_count = 0
        train_loss = 0
        train_acc_list = []
        with tqdm.tqdm(train_loader, disable=verbose==False) as t:
            for x, y in t:
                x_ = x.to(device)
                y_ = y.to(device)
                optimizer.zero_grad()
                logits = model(x_)
                loss, cor_count = criterion(logits, y_)
                loss.backward()
                train_acc += cor_count
                train_count += y.size(0)
                train_loss += loss.item()
                optimizer.step()
                t.set_postfix({'Train loss': train_loss/train_count, 'Train acc': train_acc/train_count})
            total_train_acc.append(train_acc/train_count)
    model.eval()
    valid_acc = 0
    valid_count = 0
    valid_loss = 0
    with tqdm.tqdm(test_loader, disable=verbose==False) as t:
        for x, y in t:
            x_ = x.to(device)
            y_ = y.to(device)
            logits = model(x_)
            loss, cor_count = criterion(logits, y_)
            valid_acc += cor_count
            valid_count += y.size(0)
            valid_loss += loss.item()
            t.set_postfix({'Valid loss': valid_loss/valid_count, 'Valid acc': valid_acc/valid_count})
        total_valid_acc.append(valid_acc/valid_count)
    max_val_acc = valid_acc/valid_count
    if method == 'random':
        if max_val_acc > max_random_acc:
            save_model(model, f'best_cnn_model_random')
            max_random_acc = max_val_acc
    else:
        if max_val_acc > max_bay_acc:
            save_model(model, f'best_cnn_model_bay')
            max_bay_acc = max_val_acc
    
    if max_val_acc > max_cnn_acc:
        save_model(model, f'best_cnn_model')
        max_cnn_acc = max_val_acc
    if plot:
        t = plt.plot(range(1, epochs + 1), total_train_acc, c='blue')[0]
        v = plt.plot(range(1, epochs + 1), total_valid_acc, c='green')[0]
        plt.legend([t, v], ['Train accuracy', 'Validation accuracy'])
        plt.xlabel('Epochs')
        plt.ylim(0.4, 0.8)
        plt.xticks(range(1, epochs + 1))
        plt.show()
    return max_val_acc
def get_trained_model(hidden_size, dropout_rate, learning_rate, l2_reg, num_filters, pool_size, num_mlp_layers, num_cnn_layers, stride=1, kernel_index=0, sigmoid=0, num_of_epochs=5):
    # Build Model
    kernel_size = kernel_sizes_map[int(kernel_index)-1]
    if sigmoid > 0.5:
        sigmoid = True
    else:
        sigmoid = False
    model = cnn(input_length=max_length, vocab_size=vocab_size, hidden_size=int(hidden_size), kernel_sizes=kernel_size,
                     dropout_rate=dropout_rate, num_filters=int(num_filters), num_mlp_layers=int(num_mlp_layers), output_size=output_size,
                     num_cnn_layers=int(num_cnn_layers), pool_size=int(pool_size), stride=int(stride), sigmoid=sigmoid)
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg)
    criterion = get_loss_function('BCE')
    model.to(device)
    max_val_acc = 0
    for e in range(int(num_of_epochs)):
        model.train()
        with tqdm.tqdm(train_loader, disable=True) as t:
            for x, y in t:
                x_ = x.to(device)
                y_ = y.to(device)
                optimizer.zero_grad()
                logits = model(x_)
                loss, _ = criterion(logits, y_)
                loss.backward()
                optimizer.step()
    return model

# 2. Load Data

In [3]:
train_X = np.load(f'{preprocessed_folder_path}/train_X.npy', allow_pickle=True)
train_Y = np.load(f'{preprocessed_folder_path}/train_Y.npy', allow_pickle=True)
test_X = np.load(f'{preprocessed_folder_path}/test_X.npy', allow_pickle=True)
test_Y = np.load(f'{preprocessed_folder_path}/test_Y.npy', allow_pickle=True)
output_size = train_Y.shape[1]

print(f"Train set shape: {train_X.shape}")
print(f"Test set shape: {test_X.shape}")
print(f"Output size: {output_size}")

Train set shape: (3919, 10, 9)
Test set shape: (980, 10, 9)
Output size: 1


In [4]:
train_loader = DataLoader(MyDataset(train_X, train_Y), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(MyDataset(test_X, test_Y), batch_size=batch_size)

# 3. Hyperparameter Tuning

## 3.1 Random Search

In [5]:
random_param_grid = {
    'hidden_size': [i for i in range(param_grid['hidden_size'][0], param_grid['hidden_size'][1]+1)],
    'dropout_rate': [i for i in np.arange(param_grid['dropout_rate'][0], param_grid['dropout_rate'][1]+0.1, 0.1)], 
    'learning_rate': [i for i in np.arange(param_grid['learning_rate'][0], param_grid['learning_rate'][1]+0.001, 0.001)], 
    'l2_reg': [i for i in np.arange(param_grid['l2_reg'][0], param_grid['l2_reg'][1]+0.0001, 0.0001)],
    'num_filters': [i for i in range(param_grid['num_filters'][0], param_grid['num_filters'][1]+1)],
    'pool_size': [i for i in range (param_grid['pool_size'][0], param_grid['pool_size'][1]+1)],
    'num_mlp_layers': [i for i in range(param_grid['num_mlp_layers'][0], param_grid['num_mlp_layers'][1]+1)],
    'num_cnn_layers': [i for i in range(param_grid['num_cnn_layers'][0], param_grid['num_cnn_layers'][1]+1)],
    'kernel_index': [i for i in range(param_grid['kernel_index'][0], param_grid['kernel_index'][1]+1)],
    'num_of_epochs': [i for i in range(param_grid['num_of_epochs'][0], param_grid['num_of_epochs'][1]+1)],
}
def random_search(n=10):
    best_accuracy = 0 
    best_params = {}
    tried_combinations = set()
    for i in range(n):
        print(f"Random Searching...{i+1}/{n}", flush=True)
        # Randomly get a combination
        train_param = {}
        while str(train_param) == '{}' or str(train_param) in tried_combinations:
            for param in random_param_grid:
                train_param[param] = random.choice(param_grid[param])
        train_accuracy = train_cnn(**train_param, verbose=100, method='random')
        if train_accuracy > best_accuracy:
            best_accuracy = train_accuracy
            best_params = train_param
        tried_combinations.add(str(train_param))
    return best_params

In [6]:
random_time_start = time.time()
best_random_params = random_search(50)
random_time_end = time.time()
del random_param_grid
random_training_time = (random_time_end - random_time_start)/60

Random Searching...1/50
Random Searching...2/50


100%|███████| 62/62 [00:00<00:00, 342.11it/s, Valid loss=0.0439, Valid acc=0.68]

Random Searching...3/50



100%|███████| 62/62 [00:00<00:00, 151.01it/s, Valid loss=0.0439, Valid acc=0.68]

Random Searching...4/50


...

Random Searching...46/50


Random Searching...47/50
Random Searching...48/50
Random Searching...49/50


100%|███████| 62/62 [00:00<00:00, 371.59it/s, Valid loss=0.0439, Valid acc=0.68]

Random Searching...50/50



100%|████████| 62/62 [00:00<00:00, 90.24it/s, Valid loss=0.0439, Valid acc=0.68]


In [6]:
# best_random_params = {'hidden_size': 16,
#   'dropout_rate': 0.1,
#   'learning_rate': 0.001,
#   'l2_reg': 0.0001,
#   'num_filters': 16,
#   'pool_size': 16,
#   'num_mlp_layers': 1,
#   'num_cnn_layers': 1,
#   'kernel_index': 31,
#   'num_of_epochs': 20}
# random_training_time = 20.481936212380727
best_random_params, random_training_time

({'hidden_size': 16,
  'dropout_rate': 0.1,
  'learning_rate': 0.001,
  'l2_reg': 0.0001,
  'num_filters': 16,
  'pool_size': 16,
  'num_mlp_layers': 1,
  'num_cnn_layers': 1,
  'kernel_index': 31,
  'num_of_epochs': 20},
 20.481936212380727)

In [5]:
best_random_model = load_model(f'best_cnn_model_random')
y_true, y_random_pred = predict(best_random_model, test_loader)

# accuracy
best_random_acc = accuracy_score(y_true, y_random_pred)
best_random_acc, best_random_params

NameError: name 'y_true' is not defined

In [9]:
print_classification_report(y_random_pred, y_true, ["0", "1"], title='Classification Report:')

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.99      0.88       666
           1       0.96      0.45      0.61       314

    accuracy                           0.82       980
   macro avg       0.88      0.72      0.75       980
weighted avg       0.85      0.82      0.80       980



## 3.2 Bayesian Optimization

In [29]:
bayesian_time_start = time.time()
bayesian_param_grid = {
    'hidden_size': param_grid['hidden_size'],
    'dropout_rate': param_grid['dropout_rate'], 
    'learning_rate': param_grid['learning_rate'], 
    'l2_reg': param_grid['l2_reg'],
    'num_filters': param_grid['num_filters'],
    'pool_size': param_grid['pool_size'],
    'num_mlp_layers': param_grid['num_mlp_layers'],
    'num_cnn_layers': param_grid['num_cnn_layers'],
    'kernel_index': param_grid['kernel_index'],
    'num_of_epochs': param_grid['num_of_epochs']
}
# Bounded region of parameter space
optimizer = BayesianOptimization(
    f=train_cnn,
    pbounds=bayesian_param_grid,
    verbose=2,  # verbose = 1 prints only when a maximum is obsenew_optimizerrved, verbose = 0 is silent
#     random_state=random_seed
)

optimizer.maximize(init_points=40, n_iter=160)

bayesian_time_end = time.time()
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

best_bay_params = optimizer.max['params']
bay_training_time = (bayesian_time_end - bayesian_time_start)/60

|   iter    |  target   | dropou... | hidden... | kernel... |  l2_reg   | learni... | num_cn... | num_fi... | num_ml... | num_of... | pool_size |
-------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.0       | 0.2467    | 43.9      | 10.51     | 0.0007435 | 0.006052  | 2.264     | 14.54     | 1.489     | 6.08      | 13.6      |
| 2         | 0.0       | 0.2994    | 44.98     | 12.26     | 0.0005871 | 0.003809  | 4.599     | 7.058     | 3.243     | 7.27      | 3.884     |
| 3         | 0.0       | 0.4555    | 39.92     | 23.26     | 0.0006037 | 0.007255  | 3.919     | 14.32     | 4.514     | 6.013     | 10.93     |
| 4         | 0.7316    | 0.1047    | 82.4      | 7.656     | 0.0004773 | 0.004355  | 1.406     | 13.92     | 3.744     | 5.136     | 10.9      |
| 5         | 0.7724    | 0.1714    | 99.18     | 28.06     | 0.0008061 | 0.007357  | 1.867     | 8.436     | 2.547     | 14

| 41        | 0.7765    | 0.1721    | 98.98     | 26.87     | 0.0006511 | 0.007571  | 1.62      | 7.619     | 2.747     | 14.98     | 10.06     |
| 42        | 0.7653    | 0.1119    | 84.36     | 8.378     | 0.0007739 | 0.005864  | 1.538     | 12.93     | 2.612     | 7.291     | 6.661     |
| 43        | 0.7653    | 0.2862    | 41.26     | 8.453     | 0.0009188 | 0.005285  | 1.334     | 11.42     | 3.223     | 14.67     | 15.58     |
| 44        | 0.7429    | 0.1       | 60.98     | 20.23     | 0.001     | 0.01      | 1.0       | 3.05      | 2.994     | 8.834     | 9.406     |
| 45        | 0.7837    | 0.4189    | 39.1      | 14.33     | 0.001     | 0.009248  | 1.0       | 10.71     | 1.85      | 14.05     | 11.91     |
| 46        | 0.7224    | 0.5       | 34.52     | 9.125     | 0.001     | 0.01      | 1.0       | 8.239     | 1.0       | 14.9      | 14.25     |
| 47        | 0.8041    | 0.1       | 38.2      | 12.88     | 0.001     | 0.001     | 1.0       | 16.0      | 5.0       | 15

| 75        | 0.0       | 0.5       | 85.07     | 3.3       | 0.001     | 0.01      | 5.0       | 16.0      | 5.0       | 5.0       | 2.0       |
| 76        | 0.6796    | 0.5       | 54.13     | 25.99     | 0.001     | 0.001     | 1.0       | 2.0       | 1.0       | 5.152     | 16.0      |
| 77        | 0.7694    | 0.1       | 38.42     | 22.12     | 0.0001    | 0.001     | 1.0       | 2.0       | 5.0       | 15.0      | 3.486     |
| 78        | 0.3204    | 0.1       | 46.55     | 1.0       | 0.0001    | 0.01      | 1.0       | 16.0      | 5.0       | 15.0      | 16.0      |
| 79        | 0.6959    | 0.5       | 48.22     | 31.0      | 0.0001    | 0.001     | 1.0       | 2.0       | 5.0       | 5.807     | 2.0       |
| 80        | 0.8163    | 0.1       | 98.74     | 31.0      | 0.001     | 0.001     | 1.0       | 14.8      | 1.0       | 15.0      | 6.619     |
| 81        | 0.7388    | 0.1       | 49.47     | 31.0      | 0.0001    | 0.001     | 1.0       | 13.22     | 1.0       | 15

| 109       | 0.6796    | 0.1       | 31.06     | 16.34     | 0.0001    | 0.001     | 1.0       | 2.0       | 5.0       | 5.0       | 2.0       |
| 110       | 0.6796    | 0.1       | 42.67     | 1.0       | 0.001     | 0.01      | 1.0       | 16.0      | 5.0       | 15.0      | 2.0       |
| 111       | 0.6796    | 0.5       | 30.94     | 1.0       | 0.001     | 0.01      | 1.0       | 2.0       | 1.0       | 5.0       | 2.0       |
| 112       | 0.7796    | 0.1       | 79.54     | 10.84     | 0.001     | 0.001     | 1.0       | 16.0      | 1.0       | 15.0      | 2.0       |
| 113       | 0.6796    | 0.1       | 62.23     | 3.732     | 0.0001    | 0.001     | 1.0       | 16.0      | 1.0       | 15.0      | 10.52     |
| 114       | 0.8082    | 0.1       | 43.34     | 12.02     | 0.0001    | 0.001     | 1.0       | 16.0      | 1.0       | 15.0      | 2.0       |
| 115       | 0.6796    | 0.5       | 53.84     | 1.0       | 0.001     | 0.001     | 1.0       | 16.0      | 1.0       | 15

| 143       | 0.6796    | 0.1       | 16.0      | 31.0      | 0.0001    | 0.01      | 1.0       | 2.0       | 5.0       | 15.0      | 16.0      |
| 144       | 0.698     | 0.1       | 37.94     | 12.36     | 0.001     | 0.001     | 1.0       | 2.0       | 5.0       | 15.0      | 16.0      |
| 145       | 0.8143    | 0.1       | 43.64     | 31.0      | 0.0001    | 0.001764  | 1.0       | 9.005     | 1.0       | 15.0      | 7.767     |
| 146       | 0.0       | 0.5       | 128.0     | 1.0       | 0.0001    | 0.007945  | 5.0       | 16.0      | 1.0       | 5.0       | 2.0       |
| 147       | 0.6796    | 0.1       | 34.22     | 1.0       | 0.0001    | 0.01      | 1.0       | 2.0       | 1.0       | 5.0       | 16.0      |
| 148       | 0.6796    | 0.5       | 92.34     | 7.8       | 0.0001    | 0.01      | 1.0       | 11.84     | 5.0       | 15.0      | 16.0      |
| 149       | 0.6796    | 0.1       | 42.37     | 19.05     | 0.001     | 0.01      | 1.0       | 8.942     | 5.0       | 15

| 177       | 0.6796    | 0.5       | 43.29     | 31.0      | 0.0001    | 0.001     | 1.0       | 2.0       | 5.0       | 15.0      | 16.0      |
| 178       | 0.8       | 0.1       | 63.53     | 9.558     | 0.0001    | 0.01      | 1.0       | 9.796     | 1.0       | 15.0      | 2.0       |
| 179       | 0.6796    | 0.1       | 91.25     | 31.0      | 0.001     | 0.001     | 1.0       | 16.0      | 1.0       | 5.556     | 16.0      |
| 180       | 0.6796    | 0.5       | 30.64     | 11.04     | 0.001     | 0.001     | 1.0       | 2.0       | 5.0       | 5.0       | 16.0      |
| 181       | 0.6796    | 0.1       | 116.7     | 1.0       | 0.001     | 0.001     | 1.0       | 16.0      | 5.0       | 15.0      | 16.0      |
| 182       | 0.0       | 0.1       | 16.0      | 31.0      | 0.0001    | 0.007962  | 5.0       | 2.0       | 1.0       | 5.0       | 2.0       |
| 183       | 0.6796    | 0.5       | 91.72     | 20.44     | 0.001     | 0.001     | 1.0       | 16.0      | 1.0       | 7.

In [30]:
# best_bay_params = {'dropout_rate': 0.1,
#   'hidden_size': 47.302090935166156,
#   'kernel_index': 31.0,
#   'l2_reg': 0.001,
#   'learning_rate': 0.001,
#   'num_cnn_layers': 1.0,
#   'num_filters': 16.0,
#   'num_mlp_layers': 1.0,
#   'num_of_epochs': 15.0,
#   'pool_size': 9.813986791084629}
# bay_training_time = 51.89767103592555
best_bay_params, bay_training_time

({'dropout_rate': 0.1,
  'hidden_size': 47.302090935166156,
  'kernel_index': 31.0,
  'l2_reg': 0.001,
  'learning_rate': 0.001,
  'num_cnn_layers': 1.0,
  'num_filters': 16.0,
  'num_mlp_layers': 1.0,
  'num_of_epochs': 15.0,
  'pool_size': 9.813986791084629},
 51.89767103592555)

In [31]:
best_bay_model = load_model(f'best_cnn_model_bay')
y_true, y_bay_pred = predict(best_bay_model, test_loader)

# accuracy
best_bay_acc = accuracy_score(y_true, y_bay_pred)
best_bay_acc, best_bay_params

100%|██████████████████████████████████████████| 62/62 [00:00<00:00, 147.81it/s]


(0.8346938775510204,
 {'dropout_rate': 0.1,
  'hidden_size': 47.302090935166156,
  'kernel_index': 31.0,
  'l2_reg': 0.001,
  'learning_rate': 0.001,
  'num_cnn_layers': 1.0,
  'num_filters': 16.0,
  'num_mlp_layers': 1.0,
  'num_of_epochs': 15.0,
  'pool_size': 9.813986791084629})

In [32]:
print_classification_report(y_bay_pred, y_true, ["0", "1"], title='Classification Report:')

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.98      0.89       666
           1       0.94      0.52      0.67       314

    accuracy                           0.83       980
   macro avg       0.87      0.75      0.78       980
weighted avg       0.85      0.83      0.82       980



# 4. Result Summary

In [33]:
result = {
    'random_search': {
        'training_time(mins)': random_training_time,
        'accuracy': best_random_acc,
        **get_clean_params(best_random_params)
    },
    'bayesian': {
        'training_time(mins)': bay_training_time,
        'accuracy': best_bay_acc,
        **get_clean_params(best_bay_params)
    },
}

In [34]:
result = pd.DataFrame.from_dict(result)
result

,random_search,bayesian
training_time(mins),9.135307,51.897671
accuracy,0.818367,0.834694
hidden_size,128,47
dropout_rate,0.1,0.1
learning_rate,0.001,0.001
l2_reg,0.0001,0.001
num_filters,16,16
pool_size,16,9
num_mlp_layers,1,1
num_cnn_layers,1,1
